In [1]:
# Testing notebook -- for analyzing 

In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor

import time
import random


In [46]:
# Constants
KAHOOT_URL = "https://kahoot.it/"
GAME_CODE = ""  # Replace with your game code
BASE_NAME = ''
JOIN_COUNT =  # Number of times you want to join


# Faster script -- more powerful pc required

In [47]:
def join_game(driver, game_code):
      game_code_input = driver.find_element(by=By.XPATH, value='//*[@id="game-input"]')  # Adjust XPath if needed
      game_code_input.send_keys(game_code)
      game_code_input.send_keys(Keys.ENTER)

def nickname_input(driver, url, game_code, name_suffix):
      player_name_input = driver.find_element(by=By.XPATH, value='//*[@id="nickname"]') 
      player_name_input.send_keys(f"{BASE_NAME}{name_suffix+1}")  
      player_name_input.send_keys(Keys.ENTER)


In [ ]:
# Create a new Chrome session
start_time = time.time()
chrome_options = Options()
chrome_options.add_argument("--headless") # Run without rendering Chrome
driver = webdriver.Chrome(options=chrome_options)

# execution using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=JOIN_COUNT) as executor:
    for i in range(JOIN_COUNT):
        driver.execute_script("window.open('');")  # Open a new tab
        driver.switch_to.window(driver.window_handles[-1])  # Switch to the latest tab
        driver.get(KAHOOT_URL)

    for i in range(JOIN_COUNT):
        driver.switch_to.window(driver.window_handles[i+1])
        executor.submit(join_game, driver, GAME_CODE).result()
    time.sleep(1) # For all pages to load 
    for i in range(JOIN_COUNT):
        driver.switch_to.window(driver.window_handles[i+1])
        executor.submit(nickname_input, driver, KAHOOT_URL, GAME_CODE, i+1).result()

end_time = time.time()

print(f"For {JOIN_COUNT} bots, Total time taken: {end_time - start_time} seconds")

In [ ]:
# for 40 bots, Total time taken: ~37 seconds.

In [49]:
# Red, Blue, Yellow, Green
button_xpaths = ['/html/body/div[1]/div[1]/div/div/main/div[3]/form/div/div/div[1]/button[1]',
                  '/html/body/div[1]/div[1]/div/div/main/div[3]/form/div/div/div[1]/button[2]',
                  '/html/body/div[1]/div[1]//html/body/div[1]/div[1]/div/div/main/div[3]/form/div/div/div[2]/button[2]/div/div/div[2]/button[2]']
for i in range(JOIN_COUNT):
      driver.switch_to.window(driver.window_handles[i+1])  # Switch to the latest tab

      locate_button = driver.find_element(by=By.XPATH, value=button_xpaths[0])
      locate_button.click()

      

In [51]:
driver.quit()

# Regular script

In [150]:
def join_kahoot(driver, url, game_code, name_suffix):
    try: 
      driver.get(url)

      WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="game-input"]'))
        ).send_keys(game_code, Keys.ENTER)
      
      
      # Wait for the nickname input field to be visible and interactable
      player_name_input = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.XPATH, '//*[@id="nickname"]'))
      )
      player_name_input.send_keys(f"{BASE_NAME}{name_suffix+1}")  
      player_name_input.send_keys(Keys.ENTER)

      
      print(f"Successfully joined game as {BASE_NAME}{name_suffix+1}")
    except Exception as e:
       print(f"Unexpected error in thread {name_suffix}: {e} :()")


In [ ]:
start_time = time.time()
# Create a new Chrome session
chrome_options = Options()
chrome_options.add_argument("--headless") # Run without rendering Chrome

driver = webdriver.Chrome(options=chrome_options)

for i in range(JOIN_COUNT):
    driver.execute_script("window.open('');")  # Open a new tab
    driver.switch_to.window(driver.window_handles[-1])  # Switch to the latest tab
    join_kahoot(driver, KAHOOT_URL, GAME_CODE, i+1)



end_time = time.time()

# Print the total time taken
print(f"For {JOIN_COUNT} bots, Total time taken: {end_time - start_time} seconds")
driver.quit()
print("Test completed. Quitted.")